In [ ]:
!pip install qiskit=1.4.2
!pip install qiskit_aer
!pip install qiskit_machine_learning
!pip install qiskit_algorithms
!pip install matplotlib
!pip install pylatexenc
!pip install scipy
!pip install -U scikit-learn

!git clone https://github.com/IsaVia777/atelier_qml.git

# Lab 2: Data embedding

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os 
import sys

from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap, PauliFeatureMap
from qiskit.quantum_info import Statevector
from qiskit_machine_learning.circuit.library import RawFeatureVector

SEED = 8398

In [ ]:
import sys
sys.path.insert(0, '/content/atelier_qml')

from utils import *

In [ ]:
# We will be using simulators in this lab
qasm_sim = Aer.get_backend('qasm_simulator')
sv_sim = Aer.get_backend('statevector_simulator')

### Non linearly separable 1D Database 

In [ ]:
x0,x1 = get_non_seperable_data()
nb_features = 1

In [ ]:
# Let's see what our dataset look like. Label 0 in blue, label 1 in red
plt.yticks([])
plt.scatter(x0, [0]*len(x0), color='blue')
plt.scatter(x1, [0]*len(x1), color='red')

As we can see, no straight line can separate the *blue* datapoints from the *red* ones.<br>
We say that the data is not *linearly separable*.<br>
Let's explore the idea of data embedding and quantum feature maps to make this data seperable in a different space!

## Data embedding

We can embedd classical data into quantum states by using a **quantum feature map** $\phi(\mathbf{x})$. The data is then in a new higher dimensional space. Different feauture maps exist, and we will see a few below.

 As always, the qubits are initialized in the zero state $|0\rangle$. The first layer of circuits in QML will be this data embedding layer.

### Angle embedding

We need to define a parametrized circuit. Its parameters will be used to load the data in the quantum circuit.

In [ ]:
# Create a list of `Parameter`. Since the data points have only one feature
# there will be only one parameter in this list.
x_params = [Parameter(f'x{str(i)}') for i in range(nb_features)]

# Instanciate a quantum circuit
qc = QuantumCircuit(nb_features)


for i in range(1):
    # Data encoding using the rotation RX
    qc.rx(x_params[i], i)

qc.draw('mpl')

In [ ]:
# Obtain the state vector corresponding to each points in the dataset
statevectors0 = get_statevector(qc, x0, x_params, sv_sim)
statevectors1 = get_statevector(qc, x1, x_params, sv_sim)

# Visualization of the dataset on the BLoch sphere
plot_bloch_visualization([statevectors0, statevectors1], ['b', 'r'])

### Angle data embedding of the Iris dataset

In [ ]:
x_train,y_train,x_test,y_test = get_iris(SEED)
nb_features = 4
x_train[0]

Let's define a function which takes in a quantum circuit and a feature vector, and embeds this feature vector with **angle embedding**.

In [ ]:
def angle_embedding(qc,feature_vec):  
    
    """
    Qubit - or rotation - encoding in RX gates.

    :param qc: The quantum circuit.
    :param feature_vec: The feature vector parametrizing the RX gates.
                        The number of qubit in the circuit should be equal to the number
                        of feature in the input vector. 
    :return: The quantum circuit with the embedding layer. 
    """
    if qc.num_qubits != len(feature_vec):
        raise ValueError('Number of features must match number of qubits')

    for i in range(qc.num_qubits):
        qc.rx(feature_vec[i], i)

    return qc
    

Let's start by embedding a single feature vector into a quantum state

In [ ]:
print('Features for the first datapoint:', x_train[0])

nb_qubits = nb_features
qc = QuantumCircuit(nb_qubits)

# Add the data embedding layer
qc = angle_embedding(qc, x_train[0])

qc.draw("mpl")

We can see that the values of the feauture vector are now the angles of rotations in the x axis. 

### Amplitude embedding

Let's encode this particular state:

($\frac{1}{\sqrt{2}}$, 0, 0, $\frac{1}{\sqrt{2}}$)

In [ ]:
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.draw('mpl')

Notice that this circuit creates a Bell pair!

Amplitude embedding uses no specific gates, it depends on the data vector.<br>
In some cases, like the one above, creating the correct circuit is trivial.<br>
In most cases though,it is not obvious which circuit represents the quantum state in question.

Fortunately, Qiskit can help us with this! The __[RawFeatureVector class](https://qiskit.org/documentation/machine-learning/stubs/qiskit_machine_learning.circuit.library.RawFeatureVector.html)__ loads the data into qubit amplitudes automatically. Let's try it out



In [ ]:
qc = RawFeatureVector(nb_features) 
qc.draw('mpl')

In [ ]:
qc = qc.assign_parameters(x_train[0]) 
print(x_train[0])
qc.decompose().decompose().decompose().decompose().decompose().decompose().draw('mpl')

In [ ]:
# The quantum circuit gives the right state!
Statevector.from_instruction(qc)

### Quantum feature maps with Qiskit




Qiskit also offers different feature maps: `ZFeatureMap`, the `ZZFeatureMap` and the `PauliFeatureMap`.<br>
We can draw each of these to see what exactly they do.

In [ ]:
map_z = ZFeatureMap(feature_dimension=nb_features, reps = 2)
map_z.decompose().draw('mpl', scale = 1.5)

The [P gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.PhaseGate.html) is a phase gate. 

In [ ]:
map_pauli = PauliFeatureMap(feature_dimension=4, reps=1, entanglement = 'linear')
map_pauli.decompose().draw('mpl')

In [ ]:
encode_circuit = map_pauli.assign_parameters(x_train[0])
encode_circuit.decompose().draw('mpl')

## Exercise 2: 
Embed the classical data point x = (-5, 4.5, 0.2, 1) using the ZZFeatureMap. 

In [ ]:
##Your code here##

x = None

zz_circuit = ZZFeatureMap()

zz_circuit.decompose().draw('mpl')